# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/test-bed-fltk-jerrit/fltk`
    * Push to your gcr.io repository `docker push gcr.io/test-bed-fltk-jerrit/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="fltk-flint" # changed
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL1="fltk-pool-1" # changed
EXPERIMENT_POOL2="fltk-pool-2" # changed
EXPERIMENT_POOL=$EXPERIMENT_POOL1
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet
     

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


In [3]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
     --num-nodes 4 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [4]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1011 09:21:11.908953    4448 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1011 20:16:29.941440    4354 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Tue Oct 11 20:16:33 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [5]:
EXPERIMENT_FILE="../configs/distributed_tasks/current_experiment.json"
CLUSTER_CONFIG="../configs/vision_transformer_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [85]:
helm uninstall -n test experiment-orchestrator

W1011 11:15:16.236316   13319 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found


: 1

In [6]:
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1011 20:18:15.308300    4385 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Tue Oct 11 20:18:17 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [8]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

10-11-2022 18:18:22 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-11-2022 18:18:22 root         INFO     Starting in cluster mode.
10-11-2022 18:18:22 root         INFO     Starting with experiment replication: 0 with seed: 42
10-11-2022 18:18:22 root         INFO     Starting as Orchestrator
10-11-2022 18:18:22 root         INFO     Starting Orchestrator, initializing resources....
10-11-2022 18:18:22 root         INFO     Loading in cluster configuration file
10-11-2022 18:18:22 root         INFO     Pointing configuration to in cluster configuration.
10-11-2022 18:18:22 root         INFO     Starting cluster manager
10-11-2022 18:18:22 ClusterManager INFO     Spinning up cluster manager...
10-11-2022 18:18:22 root         INFO     Starting arrival generator
10-11-2022 18:18:22 root         INFO     Starting orchestrator
10-11-2022 18:18:22 ResourceWatchDog INFO     Starting resource watchdog
10-11-2022 18:18:22 SequentialArrivalGe

In [148]:
kubectl logs -n test trainjob-0749f555-8789-4638-851a-086ca4ecaba6-master-0 

#kubectl logs -n test trainjob-40f1c0f7-50f3-45a3-b8c2-23cfbb423f6c-master-0 

#kubectl logs -n test trainjob-9db41c7d-0c78-4fed-a94e-ab7a6b7b99a3-master-0 

#kubectl logs -n test trainjob-b1339347-cbc3-405c-b577-9b897369afe1-master-0 

10-11-2022 09:33:51 root         INFO     Loading file config/configuration.fltk.json
10-11-2022 09:33:51 root         INFO     Starting in client mode
10-11-2022 09:33:51 root         INFO     Starting with host=trainjob-0749f555-8789-4638-851a-086ca4ecaba6-master-0 and port=23456
10-11-2022 09:33:51 root         INFO     Starting Creating client with 0
10-11-2022 09:33:51 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Initializing learning client
10-11-2022 09:33:51 root         INFO     Getting net: Nets.swin_flowers
10-11-2022 09:33:52 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Preparing learner model with distributed=False
10-11-2022 09:33:52 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     Starting training of model Nets.swin_flowers with batch size 25 and max epochs 90.
10-11-2022 09:33:58 Client-0-0749f555-8789-4638-851a-086ca4ecaba6 INFO     [1,     0] loss: 0.096
/opt/federation-lab/fltk/nets/util/evaluation.py:8: RuntimeWarning: invalid value encoun

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [145]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ./logging

error: error executing jsonpath "{.items[0].metadata.name}": Error executing template: array index out of bounds: index 0, length 0. Printing more information for debugging the template:
	template was:
		{.items[0].metadata.name}
	object given to jsonpath engine was:
		map[string]interface {}{"apiVersion":"v1", "items":[]interface {}{}, "kind":"List", "metadata":map[string]interface {}{"resourceVersion":"", "selfLink":""}}


error: filespec must match the canonical format: [[namespace/]pod:]file/path


: 1

# Cleanup

## Removing orchestrator

In [9]:
helm uninstall -n test experiment-orchestrator

W1011 20:29:44.621416    6455 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [10]:
helm uninstall extractor -n test

W1011 20:32:32.534767    8031 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [11]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

pytorchjob.kubeflow.org "trainjob-99544cf3-2501-41e9-b7b1-468582f9cccd" deleted


In [77]:
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-jerrit/zones/us-central1-c/clusters/fltk-testbed-cluster].
